<a href="https://colab.research.google.com/github/ggsgranados/TelecomX-LATAM-Parte-2/blob/main/Telecom_X_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports necesarios

In [78]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

##📌 Extracción del archivo tratado

In [79]:
# Importando el JSON y convirtiendolo a un DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/ggsgranados/TelecomX-LATAM-Parte-2/refs/heads/main/df_normalizado.csv')

In [80]:
df.head(5)

Churn  SeniorCitizen  tenure  Charges.Monthly  Charges.Total  \
0    0.0              0       9             65.6         593.30   
1    0.0              0       9             59.9         542.40   
2    1.0              0       4             73.9         280.85   
3    1.0              1      13             98.0        1237.85   
4    1.0              1       3             83.9         267.40   

   gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  ...  \
0              1            0           0            1              0  ...   
1              0            1           1            0              1  ...   
2              0            1           1            0              1  ...   
3              0            1           0            1              1  ...   
4              1            0           0            1              1  ...   

   Contract_One year  Contract_Two year  PaperlessBilling_No  \
0                  1                  0                    0   
1                  0                  0                    1   
2                  0                  0                    0   
3                  0                  0                    0   
4                  0                  0                    0   

   PaperlessBilling_Yes  PaymentMethod_Bank transfer (automatic)  \
0                     1                                        0   
1                     0                                        0   
2                     1                                        0   
3                     1                                        0   
4                     1                                        0   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                      0                               0   
1                                      0                               0   
2                                      0                               1   
3                                      0                               1   
4                                      0                               0   

   PaymentMethod_Mailed check  Cuentas_Diarias     Churn_Label  
0                           1         2.155059      Permaneció  
1                           1         1.967806      Permaneció  
2                           0         2.427727  Se dio de baja  
3                           0         3.219448  Se dio de baja  
4                           1         2.756242  Se dio de baja  

[5 rows x 48 columns]

##❌ Eliminación de columnas irrelevantes

In [81]:
def remove_negative_dummies(df):
    columns_to_drop = []
    for col in df.columns:
        # Buscar columnas que terminen en '_No' o '_no'
        if col.endswith('_No') or col.endswith('_no'):
            # Identificar la columna correspondiente con lógica positiva
            positive_col_name_yes = col.replace('_No', '_Yes').replace('_no', '_yes')
            positive_col_name_blank = col.replace('_No', '_ ').replace('_no', '_ ')
            positive_col_name_single = col.replace('_No', '').replace('_no', '')

            # Verificar si la columna de lógica positiva existe
            if positive_col_name_yes in df.columns or positive_col_name_blank in df.columns or positive_col_name_single in df.columns:
                columns_to_drop.append(col)

    if columns_to_drop:
        print(f"\nEliminando las siguientes columnas con lógica negativa: {columns_to_drop}")
        df = df.drop(columns=columns_to_drop)
    else:
        print("\nNo se encontraron columnas con lógica negativa ('_No' o '_no') para eliminar.")

    return df

# Aplicar la función al DataFrame
df_normalizado = remove_negative_dummies(df)

print("\nColumnas después de eliminar las de lógica negativa:")
print(df_normalizado.columns.tolist())
print("\nEstado final del DataFrame:")
print(df_normalizado.info())


Eliminando las siguientes columnas con lógica negativa: ['Partner_No', 'Dependents_No', 'PhoneService_No', 'MultipleLines_No', 'OnlineSecurity_No', 'OnlineBackup_No', 'DeviceProtection_No', 'TechSupport_No', 'StreamingTV_No', 'StreamingMovies_No', 'PaperlessBilling_No']

Columnas después de eliminar las de lógica negativa:
['Churn', 'SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'gender_Female', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contrac

In [82]:
# Columnas a eliminar
columns_to_drop = ['gender_Female', 'Churn_Label']

# Verificar si las columnas existen en el DataFrame antes de eliminarlas
existing_columns_to_drop = [col for col in columns_to_drop if col in df_normalizado.columns]

if existing_columns_to_drop:
    print(f"\nEliminando las siguientes columnas: {existing_columns_to_drop}")
    df_normalizado = df_normalizado.drop(columns=existing_columns_to_drop)
else:
    print("\nLas columnas 'gender_Female' y 'Churn_Label' no se encontraron en el DataFrame.")

print("\nColumnas después de eliminar gender_Female y Churn_Label:")
print(df_normalizado.columns.tolist())
print("\nEstado final del DataFrame:")
print(df_normalizado.info())



Eliminando las siguientes columnas: ['gender_Female', 'Churn_Label']

Columnas después de eliminar gender_Female y Churn_Label:
['Churn', 'SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Cuentas

## ✅ Verificación de la proporción de cancelación (Churn)

In [83]:
churn_proportion = df_normalizado['Churn'].value_counts(normalize=True)
print("\nProporción de clientes que cancelaron (Churn = 1) vs. activos (Churn = 0) :")
churn_proportion



Proporción de clientes que cancelaron (Churn = 1) vs. activos (Churn = 0) :


Churn
0.0    0.73463
1.0    0.26537
Name: proportion, dtype: float64

## ⚖️ Balanceo de clases

In [84]:
# Verificando valores NaN
if df_normalizado['Churn'].isnull().any():
    print("\Advertencia: Velores NaN encontrados. Eliminando valores NaN")
    df_normalizado.dropna(subset=['Churn'], inplace=True)

# Separando clases
df_majority = df_normalizado[df_normalizado.Churn==0]
df_minority = df_normalizado[df_normalizado.Churn==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=123) # estado aleatorio para reproducibilidad

# combinando clases
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Mostrando nuevas clases
print("\nProporción de clases después del balanceo:")
print(df_balanced.Churn.value_counts(normalize=True))

print("\nEstado final del DataFrame balanceado:")
print(df_balanced.info())

# Asignando el balanceo
df_normalizado = df_balanced

\Advertencia: Velores NaN encontrados. Eliminando valores NaN

Proporción de clases después del balanceo:
Churn
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64

Estado final del DataFrame balanceado:
<class 'pandas.core.frame.DataFrame'>
Index: 10348 entries, 0 to 6827
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Churn                                    10348 non-null  float64
 1   SeniorCitizen                            10348 non-null  int64  
 2   tenure                                   10348 non-null  int64  
 3   Charges.Monthly                          10348 non-null  float64
 4   Charges.Total                            10348 non-null  float64
 5   gender_Male                              10348 non-null  int64  
 6   Partner_Yes                              10348 non-null  int64  
 7   Dependents_Yes                           10348 non-null

In [85]:
df_normalizado.head(5)

Churn  SeniorCitizen  tenure  Charges.Monthly  Charges.Total  gender_Male  \
0    0.0              0       9            65.60         593.30            0   
1    0.0              0       9            59.90         542.40            1   
5    0.0              0       9            69.40         571.45            0   
6    0.0              1      71           109.70        7904.25            0   
7    0.0              0      63            84.65        5377.80            1   

   Partner_Yes  Dependents_Yes  PhoneService_Yes  \
0            1               1                 1   
1            0               0                 1   
5            0               1                 1   
6            1               0                 1   
7            1               0                 1   

   MultipleLines_No phone service  ...  StreamingMovies_Yes  \
0                               0  ...                    0   
1                               0  ...                    1   
5                               0  ...                    1   
6                               0  ...                    1   
7                               0  ...                    0   

   Contract_Month-to-month  Contract_One year  Contract_Two year  \
0                        0                  1                  0   
1                        1                  0                  0   
5                        1                  0                  0   
6                        0                  0                  1   
7                        0                  0                  1   

   PaperlessBilling_Yes  PaymentMethod_Bank transfer (automatic)  \
0                     1                                        0   
1                     0                                        0   
5                     1                                        0   
6                     1                                        1   
7                     1                                        0   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                      0                               0   
1                                      0                               0   
5                                      1                               0   
6                                      0                               0   
7                                      1                               0   

   PaymentMethod_Mailed check  Cuentas_Diarias  
0                           1         2.155059  
1                           1         1.967806  
5                           0         2.279895  
6                           0         3.603811  
7                           0         2.780880  

[5 rows x 35 columns]

## 🔧 Normalización/Estandarización

In [86]:
# ## 🤖 Preparación de datos para el modelo (Normalización/Estandarización)

# Evaluar la necesidad de normalizar o estandarizar los datos.
# La estandarización (usando StandardScaler) es más adecuada cuando los datos siguen una distribución gaussiana.
# La normalización (usando MinMaxScaler) es más adecuada cuando se necesita que los datos estén en un rango específico [0, 1]
# o cuando los datos no siguen una distribución gaussiana y pueden tener valores atípicos.
# Dado que algunas características pueden no seguir una distribución gaussiana y para mantener los datos en un rango similar,
# se utilizará MinMaxScaler.

print("\nPreparando datos para el modelo: Normalización/Estandarización")

# Separar características (X) y variable objetivo (y)
X = df_normalizado.drop('Churn', axis=1)
y = df_normalizado['Churn']

# Identificar columnas numéricas para escalar (excluir las dummy/binarias)
# Las columnas dummy ya están en el rango [0, 1], por lo que no necesitan normalización.
# Suponemos que las columnas numéricas son aquellas con dtype float64 o int64, excluyendo la columna 'Churn' que ya se separó.
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()

print(f"\nColumnas numéricas a escalar: {numeric_cols}")

# Inicializar el escalador (MinMaxScaler)
scaler = MinMaxScaler()

# Aplicar normalización solo a las columnas numéricas
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

print("\nDatos escalados (MinMaxScaler) aplicados a columnas numéricas.")

print("\nEstado final del DataFrame después de la normalización:")
print(X.info())


Preparando datos para el modelo: Normalización/Estandarización

Columnas numéricas a escalar: ['SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Cuentas_Diarias']

Datos escalados (MinMaxScaler) 

In [87]:
# Mostrar una muestra de los datos escalados
print("\nPrimeras 5 filas del DataFrame con datos escalados:")
X.head(5)


Primeras 5 filas del DataFrame con datos escalados:


SeniorCitizen    tenure  Charges.Monthly  Charges.Total  gender_Male  \
0            0.0  0.125000         0.471144       0.068315          0.0   
1            0.0  0.125000         0.414428       0.062454          1.0   
5            0.0  0.125000         0.508955       0.065799          0.0   
6            1.0  0.986111         0.909950       0.910125          0.0   
7            0.0  0.875000         0.660697       0.619220          1.0   

   Partner_Yes  Dependents_Yes  PhoneService_Yes  \
0          1.0             1.0               1.0   
1          0.0             0.0               1.0   
5          0.0             1.0               1.0   
6          1.0             0.0               1.0   
7          1.0             0.0               1.0   

   MultipleLines_No phone service  MultipleLines_Yes  ...  \
0                             0.0                0.0  ...   
1                             0.0                1.0  ...   
5                             0.0                0.0  ...   
6                             0.0                0.0  ...   
7                             0.0                1.0  ...   

   StreamingMovies_Yes  Contract_Month-to-month  Contract_One year  \
0                  0.0                      0.0                1.0   
1                  1.0                      1.0                0.0   
5                  1.0                      1.0                0.0   
6                  1.0                      0.0                0.0   
7                  0.0                      0.0                0.0   

   Contract_Two year  PaperlessBilling_Yes  \
0                0.0                   1.0   
1                0.0                   0.0   
5                0.0                   1.0   
6                1.0                   1.0   
7                1.0                   1.0   

   PaymentMethod_Bank transfer (automatic)  \
0                                      0.0   
1                                      0.0   
5                                      0.0   
6                                      1.0   
7                                      0.0   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                    0.0                             0.0   
1                                    0.0                             0.0   
5                                    1.0                             0.0   
6                                    0.0                             0.0   
7                                    1.0                             0.0   

   PaymentMethod_Mailed check  Cuentas_Diarias  
0                         1.0         0.471144  
1                         1.0         0.414428  
5                         0.0         0.508955  
6                         0.0         0.909950  
7                         0.0         0.660697  

[5 rows x 34 columns]